# LOAD DATASET 

In [27]:
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import OneClassSVM
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, classification_report, f1_score
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np 


In [28]:
dataset = pd.read_csv("Obfuscated-MalMem2022.csv")

In [29]:
dataset.head()

,Category,pslist.nproc,pslist.nppid,pslist.avg_threads,pslist.nprocs64bit,pslist.avg_handlers,dlllist.ndlls,dlllist.avg_dlls_per_proc,handles.nhandles,handles.avg_handles_per_proc,...,svcscan.kernel_drivers,svcscan.fs_drivers,svcscan.process_services,svcscan.shared_process_services,svcscan.interactive_process_services,svcscan.nactive,callbacks.ncallbacks,callbacks.nanonymous,callbacks.ngeneric,Class
0,Benign,45,17,10.555556,0,202.844444,1694,38.500000,9129,212.302326,...,221,26,24,116,0,121,87,0,8,Benign
1,Benign,47,19,11.531915,0,242.234043,2074,44.127660,11385,242.234043,...,222,26,24,118,0,122,87,0,8,Benign
2,Benign,40,14,14.725000,0,288.225000,1932,48.300000,11529,288.225000,...,222,26,27,118,0,120,88,0,8,Benign
3,Benign,32,13,13.500000,0,264.281250,1445,45.156250,8457,264.281250,...,222,26,27,118,0,120,88,0,8,Benign
4,Benign,42,16,11.452381,0,281.333333,2067,49.214286,11816,281.333333,...,222,26,24,118,0,124,87,0,8,Benign


In [30]:
## Drop Category, not needed. 

In [31]:
df = dataset.drop(['Category'], axis=1)

In [32]:
## make class 0 and 1 instead of malware and benign 
## malware = 1
## Bennign = 0 


In [33]:
df['Class'] = df['Class'].replace(['Malware', 'Benign'], [1, 0]) 

In [34]:
##Split dataset into X and Y values
## Because these models are unsupervised, labels are not needed for training 
# only used for evaluation at the end. 

In [35]:
#### using a tree based model to determine feature importance 
#### Using Random Forest 
#### Choosing top 10 features based on feature importance 

In [36]:
RF_classifier = RandomForestClassifier(n_estimators = 100)
RF_classifier.fit(Xtrain, ytrain)
important_features = RF_classifier.feature_importances_
important_features.sort()

for feature, importance in zip(dataset.columns, important_features):
    print(f"{feature}: {importance}")

Category: 0.0
pslist.nproc: 0.0
pslist.nppid: 0.0
pslist.avg_threads: 0.0
pslist.nprocs64bit: 0.0
pslist.avg_handlers: 0.0
dlllist.ndlls: 0.0
dlllist.avg_dlls_per_proc: 1.0453647421476357e-06
handles.nhandles: 2.3782607970954434e-05
handles.avg_handles_per_proc: 2.6774261814657493e-05
handles.nport: 3.576431697524416e-05
handles.nfile: 3.854514052953824e-05
handles.nevent: 5.504202847931035e-05
handles.ndesktop: 5.505398118862935e-05
handles.nkey: 8.034859452306122e-05
handles.nthread: 8.097613270952916e-05
handles.ndirectory: 0.00010077864991193821
handles.nsemaphore: 0.00010182461602036595
handles.ntimer: 0.00014238674672896055
handles.nsection: 0.00014775165871054192
handles.nmutant: 0.000312551716155908
ldrmodules.not_in_load: 0.00034295895559692584
ldrmodules.not_in_init: 0.0003810302052152295
ldrmodules.not_in_mem: 0.0004097119212238784
ldrmodules.not_in_load_avg: 0.000426480501946189
ldrmodules.not_in_init_avg: 0.0005260688744785127
ldrmodules.not_in_mem_avg: 0.00056255316499035

In [37]:
dataset_top_10_features = pd.DataFrame()

In [38]:
dataset_top_10_features['callbacks.nanonymous'] = df['callbacks.nanonymous']
dataset_top_10_features['callbacks.ncallbacks'] = df['callbacks.ncallbacks']
dataset_top_10_features['svcscan.nactive'] = df['svcscan.nactive']
dataset_top_10_features['svcscan.interactive_process_services'] = df['svcscan.interactive_process_services']
dataset_top_10_features['svcscan.shared_process_services'] = df['svcscan.shared_process_services']
dataset_top_10_features['svcscan.fs_drivers'] = df['svcscan.fs_drivers']
dataset_top_10_features['svcscan.kernel_drivers'] = df['svcscan.kernel_drivers']
dataset_top_10_features['svcscan.nservices'] = df['svcscan.nservices']
dataset_top_10_features['modules.nmodules'] = df['modules.nmodules']
dataset_top_10_features['psxview.not_in_deskthrd_false_avg'] = df['psxview.not_in_deskthrd_false_avg']
dataset_top_10_features['Class'] = df['Class']

In [39]:
X = dataset_top_10_features.iloc[:,:-1].values
y = dataset_top_10_features.iloc[:,-1].values

In [40]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X,y,test_size=0.2, random_state=42)

In [41]:
dataset_top_10_features.head()

,callbacks.nanonymous,callbacks.ncallbacks,svcscan.nactive,svcscan.interactive_process_services,svcscan.shared_process_services,svcscan.fs_drivers,svcscan.kernel_drivers,svcscan.nservices,modules.nmodules,psxview.not_in_deskthrd_false_avg,Class
0,0,87,121,0,116,26,221,389,138,0.191489,0
1,0,87,122,0,118,26,222,392,138,0.127660,0
2,0,88,120,0,118,26,222,395,137,0.125000,0
3,0,88,120,0,118,26,222,395,138,0.187500,0
4,0,87,124,0,118,26,222,392,138,0.217391,0


# Training And Evaluating Isolation Forest

In [42]:
Isolation_Forest_model=IsolationForest(max_samples= 100, n_estimators= 200, contamination=0.5, bootstrap = True)

In [43]:
Isolation_Forest_model.fit(Xtrain)

IsolationForest(bootstrap=True, contamination=0.5, max_samples=100,
                n_estimators=200)

In [44]:
predictions = Isolation_Forest_model.predict(Xtest)

In [45]:
print(predictions)

[-1  1  1 ...  1  1  1]


In [46]:
## printing the predictions
## model predicted 5904 anomalies and 5816 normal instances 

In [47]:
print(pd.value_counts(predictions))

-1    5872
 1    5848
dtype: int64


In [48]:
## map the predictions to 0 and 1 from -1 and 1
## map anomalies to 1 and normal instances to 0

In [49]:
predictions[predictions== 1] = 0
predictions[predictions== -1] = 1

In [50]:
print(pd.value_counts(predictions))

1    5872
0    5848
dtype: int64


In [51]:
## start evaluation. 
## Classification Report
## confusion Matrix 
## area under curve 

In [52]:
Isolation_forest_classification_report = classification_report(predictions, ytest)

In [53]:
print(Isolation_forest_classification_report)

              precision    recall  f1-score   support

           0       0.46      0.45      0.46      5848
           1       0.46      0.47      0.46      5872

    accuracy                           0.46     11720
   macro avg       0.46      0.46      0.46     11720
weighted avg       0.46      0.46      0.46     11720



In [54]:
Isolation_forest_confusion_matrix = confusion_matrix(predictions, ytest)
print(Isolation_forest_confusion_matrix)

[[2649 3199]
 [3141 2731]]


# Training And Evaluating Local Outlier Factor

In [55]:
Local_outlier_factor = LocalOutlierFactor(n_neighbors = 100, algorithm='auto', contamination=0.5, novelty=True)

In [56]:
Local_outlier_factor.fit(Xtrain)

LocalOutlierFactor(contamination=0.5, n_neighbors=100, novelty=True)

In [57]:
LOF_predictions = Local_outlier_factor.predict(Xtest)

In [58]:
LOF_predictions[LOF_predictions== 1] = 0
LOF_predictions[LOF_predictions== -1] = 1

print(pd.value_counts(LOF_predictions))

0    6955
1    4765
dtype: int64


In [59]:
LOF_classification_report = classification_report(LOF_predictions, ytest)
print(LOF_classification_report)

              precision    recall  f1-score   support

           0       0.62      0.52      0.56      6955
           1       0.43      0.54      0.48      4765

    accuracy                           0.53     11720
   macro avg       0.53      0.53      0.52     11720
weighted avg       0.55      0.53      0.53     11720



In [60]:
LOF_confusion_matrix = confusion_matrix(LOF_predictions, ytest)
print(LOF_confusion_matrix)

[[3599 3356]
 [2191 2574]]


# Training And Evaluating ONE-CLASS SUPPORT VECTOR MACHINE (SVM)

In [61]:
one_class_svm = OneClassSVM()

In [62]:
one_class_svm.fit(Xtrain)

OneClassSVM()

In [63]:
one_class_svm_predictions=one_class_svm.predict(Xtest)

In [64]:
one_class_svm_predictions[one_class_svm_predictions== 1] = 0
one_class_svm_predictions[one_class_svm_predictions== -1] = 1

print(pd.value_counts(one_class_svm_predictions))

1    6607
0    5113
dtype: int64


In [65]:
SVM_classification_report = classification_report(one_class_svm_predictions, ytest)
print(SVM_classification_report)

              precision    recall  f1-score   support

           0       0.22      0.24      0.23      5113
           1       0.35      0.31      0.33      6607

    accuracy                           0.28     11720
   macro avg       0.28      0.28      0.28     11720
weighted avg       0.29      0.28      0.29     11720



In [66]:
SVM_confusion_matrix = confusion_matrix(one_class_svm_predictions, ytest)
print(SVM_confusion_matrix)

[[1251 3862]
 [4539 2068]]


# Training And Evaluating ONE-CLASS SVM WITH STOCHASTIC GRADIENT DESCENT

In [67]:
# Initialize model parameters
w = np.zeros(Xtrain.shape[1])
b = 0

# Set SGD parameters
learning_rate = 0.01
epochs = 100 

In [68]:
for epoch in range(epochs):
    for i in range(len(Xtrain)):
        x = Xtrain[i]

        # Calculate the loss and gradient
        loss = np.maximum(0, 1 - np.dot(w, x) + b)
        gradient = -x if loss > 0 else 0

        # Update the model parameters
        w -= learning_rate * gradient
        b -= learning_rate

In [69]:
for epoch in range(epochs):
    for i in range(len(X_train)):
        x = X_train[i]

        # Calculate the loss and gradient
        loss = np.maximum(0, 1 - np.dot(w, x) + b)
        gradient = -x if loss > 0 else 0

        # Update the model parameters
        w -= learning_rate * gradient
        b -= learning_rate

NameError: name 'X_train' is not defined

In [ ]:
def predict(x):
    return np.sign(np.dot(w, x) + b)


In [ ]:
y_pred = np.array([predict(x) for x in Xtest])

In [ ]:
precision = precision_score(ytest, y_pred)
recall = recall_score(ytest, y_pred)
f1 = f1_score(ytest, y_pred)
accu
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

In [ ]:
SVM_SGD_classification_report = classification_report(y_pred, ytest)
print(SVM_SGD_classification_report)